# AeroClub RecSys 2025 - XGBoost, LightGBM and combined Ensemble methods

This notebook implements a ranking approach using XGBoost, LightGBM or an ensemble and Polars for the AeroClub recommendation challenge.

In [ ]:
!pip install shap

In [ ]:
!pip install optuna
!pip install optuna-integration[lightgbm]

In [ ]:
!pip install -U xgboost
!pip install -U polars
!pip install -U lightgbm
!pip install -U sklearn

In [ ]:
import polars as pl
import numpy as np
import matplotlib.pyplot as plt
import time
import xgboost as xgb
import lightgbm as lgb


RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

In [ ]:
# Load data
train = pl.read_parquet('/kaggle/input/aeroclub-recsys-2025/train.parquet').drop('__index_level_0__')
test = pl.read_parquet('/kaggle/input/aeroclub-recsys-2025/test.parquet').drop('__index_level_0__').with_columns(pl.lit(0, dtype=pl.Int64).alias("selected"))

data_raw = pl.concat((train, test))

## Helpers

In [ ]:
def hitrate_at_3(y_true, y_pred, groups):
    df = pl.DataFrame({
        'group': groups,
        'pred': y_pred,
        'true': y_true
    })
    
    return (
        df.filter(pl.col("group").count().over("group") > 10)
        .sort(["group", "pred"], descending=[False, True])
        .group_by("group", maintain_order=True)
        .head(3)
        .group_by("group")
        .agg(pl.col("true").max())
        .select(pl.col("true").mean())
        .item()
    )

## Feature Engineering

In [ ]:
df = data_raw.clone()

# More efficient duration to minutes converter
def dur_to_min(col):
    # Extract days and time parts in one pass
    days = col.str.extract(r"^(\d+)\.", 1).cast(pl.Int64).fill_null(0) * 1440
    time_str = pl.when(col.str.contains(r"^\d+\.")).then(col.str.replace(r"^\d+\.", "")).otherwise(col)
    hours = time_str.str.extract(r"^(\d+):", 1).cast(pl.Int64).fill_null(0) * 60
    minutes = time_str.str.extract(r":(\d+):", 1).cast(pl.Int64).fill_null(0)
    return (days + hours + minutes).fill_null(0)
    

dur_cols = ["legs0_duration", "legs1_duration"] + [f"legs{l}_segments{s}_duration" for l in (0, 1) for s in range(0, 4)]
dur_exprs = [dur_to_min(pl.col(c)).alias(c) for c in dur_cols if c in df.columns]

if dur_exprs:
    df = df.with_columns(dur_exprs)


mc_cols = [f'legs{l}_segments{s}_marketingCarrier_code' for l in (0, 1) for s in range(4)]
mc_exists = [col for col in mc_cols if col in df.columns]

df = df.with_columns([
    
        (pl.col("totalPrice") / (pl.col("taxes") + 1)).alias("price_per_tax"),
        (pl.col("taxes") / (pl.col("totalPrice") + 1)).alias("tax_rate"),
        pl.col("totalPrice").log1p().alias("log_price"),
    
        (pl.col("legs0_duration").fill_null(0) + pl.col("legs1_duration").fill_null(0)).alias("total_duration"),
    
        pl.when(pl.col("legs1_duration").fill_null(0) > 0)
            .then(pl.col("legs0_duration") / (pl.col("legs1_duration") + 1))
            .otherwise(1.0).alias("duration_ratio"),
    
        (pl.col("legs1_duration").is_null() | 
         (pl.col("legs1_duration") == 0) | 
         pl.col("legs1_segments0_departureFrom_airport_iata").is_null()).cast(pl.Int32).alias("is_one_way"),
    
        (pl.sum_horizontal(pl.col(col).is_not_null().cast(pl.UInt8) for col in mc_exists) 
         if mc_exists else pl.lit(0)).alias("segments_count"),
    
        (pl.col("frequentFlyer").fill_null("").str.count_matches("/") + 
         (pl.col("frequentFlyer").fill_null("") != "").cast(pl.Int32)).alias("n_ff_programs"),

        pl.col("corporateTariffCode").is_not_null().cast(pl.Int32).alias("has_corporate_tariff"),
        (pl.col("pricingInfo_isAccessTP") == 1).cast(pl.Int32).alias("has_access_tp"),

        (pl.col("legs0_segments0_baggageAllowance_quantity").fill_null(0) + 
         pl.col("legs1_segments0_baggageAllowance_quantity").fill_null(0)).alias("baggage_total"),
        (pl.col("miniRules0_monetaryAmount").fill_null(0) + 
         pl.col("miniRules1_monetaryAmount").fill_null(0)).alias("total_fees"),
        
        pl.col("searchRoute").is_in(["MOWLED/LEDMOW", "LEDMOW/MOWLED", "MOWLED", "LEDMOW", "MOWAER/AERMOW"])
            .cast(pl.Int32).alias("is_popular_route"),
    
        pl.mean_horizontal(["legs0_segments0_cabinClass", "legs1_segments0_cabinClass"]).alias("avg_cabin_class"),
        (pl.col("legs0_segments0_cabinClass").fill_null(0) - 
         pl.col("legs1_segments0_cabinClass").fill_null(0)).alias("cabin_class_diff"),
])


total_duration_expr = pl.sum_horizontal(pl.col(r"^legs\d+_duration$")).fill_null(0)

total_flight_time_expr = pl.sum_horizontal(pl.col(r"^legs\d+_segments\d+_duration$")).fill_null(0)

total_wait_time_expr = total_duration_expr - total_flight_time_expr

wait_time_ratio_expr = pl.when(total_duration_expr > 0)\
                         .then(total_wait_time_expr / total_duration_expr)\
                         .otherwise(0.0)


df = df.with_columns(
    total_wait_time = total_wait_time_expr,
    wait_time_ratio = wait_time_ratio_expr,
)


df = df.with_columns(
    days_to_departure=(pl.col("legs0_departureAt").str.to_datetime() - pl.col("requestDate")).dt.total_days()
).with_columns(
    last_minute_flight=((pl.col("days_to_departure") <= 30).cast(pl.Int8)),
    flight_advance_bucket=pl.when(pl.col("days_to_departure").is_between(0, 7))
      .then(pl.lit(0, dtype=pl.UInt8))
    .when(pl.col("days_to_departure").is_between(8, 30))
      .then(pl.lit(1, dtype=pl.UInt8))
    .when(pl.col("days_to_departure").is_between(31, 90))
      .then(pl.lit(2, dtype=pl.UInt8))
    .otherwise(pl.lit(3, dtype=pl.UInt8))
)

carrier_cols = df.select(pl.col("^legs.*_marketingCarrier_code$")).columns

df_with_ratio = df.with_columns(
    matching_segments_count = pl.sum_horizontal(
        [
            pl.col(c).is_in(pl.col("frequentFlyer").str.split("/"))
            for c in carrier_cols
        ]
    ).fill_null(0)

).with_columns(
    ff_match_ratio = (
        pl.col("matching_segments_count") / pl.col("segments_count")
    ).fill_nan(0)
)


def process_fare_rule(df, rule_index, price_col):
    status_col = f"miniRules{rule_index}_statusInfos"
    money_col = f"miniRules{rule_index}_monetaryAmount"
    percent_col = f"miniRules{rule_index}_percentage"


    df_processed = df.with_columns(
        pl.when(pl.col(status_col) == 0)
          .then(pl.col(price_col))
          .otherwise(pl.col(money_col))
          .alias(money_col),

        pl.when(pl.col(status_col) == 0)
          .then(pl.lit(100.0))
          .otherwise(pl.col(percent_col))
          .alias(percent_col)

    ).with_columns(
        pl.when(pl.col(percent_col).is_null())
          .then((pl.col(money_col) / pl.col(price_col)) * 100)
          .otherwise(pl.col(percent_col))
          .alias(percent_col)

    ).with_columns(
        pl.when(pl.col(money_col).is_null())
          .then((pl.col(percent_col) / 100) * pl.col(price_col))
          .otherwise(pl.col(money_col))
          .alias(money_col)
    )

    return df_processed


df = process_fare_rule(df, rule_index=0, price_col="totalPrice")
df = process_fare_rule(df, rule_index=1, price_col="totalPrice")

df = (
    df.join(
        train.group_by('legs0_segments0_marketingCarrier_code').agg(pl.mean('selected').alias('carrier0_pop')),
        on='legs0_segments0_marketingCarrier_code', 
        how='left'
    )
    .join(
        train.group_by('legs1_segments0_marketingCarrier_code').agg(pl.mean('selected').alias('carrier1_pop')),
        on='legs1_segments0_marketingCarrier_code', 
        how='left'
    )
    .with_columns([
        pl.col('carrier0_pop').fill_null(0.0),
        pl.col('carrier1_pop').fill_null(0.0),
    ])
)

df = df.with_columns([
    pl.when(pl.col('legs1_segments0_marketingCarrier_code').is_null())
      .then(pl.col('carrier0_pop'))
      .otherwise(pl.col('carrier0_pop') * pl.col('carrier1_pop'))
      .alias('carrier_pop_combined')
])


for leg_idx in range(2):
    for seg_idx in range(4):
        
        qty_col = f"legs_{leg_idx}_segments_{seg_idx}_baggageAllowance_quantity"
        type_col = f"legs_{leg_idx}_segments_{seg_idx}_baggageAllowance_weightMeasurementType"
        
        if qty_col in df.columns:
            
            df = df.with_columns(
                pl.when(pl.col(type_col) == 0)
                  .then(pl.col(qty_col).fill_null(0) * 20)
                  .otherwise(pl.col(qty_col).fill_null(0))
                  .alias(qty_col) # Overwrite the original quantity column
            )


cols_to_drop = df.select(pl.col("^.*weightMeasurementType$")).columns
df = df.drop(cols_to_drop)


seg_exprs = []
for leg in (0, 1):
    seg_cols = [f"legs{leg}_segments{s}_duration" for s in range(4) if f"legs{leg}_segments{s}_duration" in df.columns]
    if seg_cols:
        seg_exprs.append(
            pl.sum_horizontal(pl.col(c) != 0 for c in seg_cols)
                .cast(pl.Int32).alias(f"n_segments_leg{leg}")
        )
    else:
        seg_exprs.append(pl.lit(0).cast(pl.Int32).alias(f"n_segments_leg{leg}"))

df = df.with_columns(seg_exprs)


df = df.with_columns([
    (pl.col("n_segments_leg0") == 1).cast(pl.Int32).alias("is_direct_leg0"),
    pl.when(pl.col("is_one_way") == 1).then(0)
        .otherwise((pl.col("n_segments_leg1") == 1).cast(pl.Int32)).alias("is_direct_leg1"),
    pl.col("Id").count().over("ranker_id").alias("group_size"),
])

df = df.with_columns([
    (pl.col("is_direct_leg0") & pl.col("is_direct_leg1")).cast(pl.Int32).alias("both_direct"),
    ((pl.col("isVip") == 1) | (pl.col("n_ff_programs") > 0)).cast(pl.Int32).alias("is_vip_freq"),
    (pl.col("baggage_total") > 0).cast(pl.Int32).alias("has_baggage"),
    (pl.col("total_fees") > 0).cast(pl.Int32).alias("has_fees"),
    (pl.col("total_fees") / (pl.col("totalPrice") + 1)).alias("fee_rate"),
    pl.col("group_size").log1p().alias("group_size_log"),
    (pl.col("total_duration") == (pl.col("total_duration")).min().over("ranker_id")).cast(pl.Int32).alias("is_fastest"),
    pl.col("legs0_duration") == pl.col("legs0_duration").min().over("ranker_id").cast(pl.Int32).alias("is_leg0_fastest"),
    (pl.col("segments_count") == pl.col("segments_count").min().over("ranker_id")).cast(pl.Int32).alias("is_min_segments"),
])


if "legs0_segments0_marketingCarrier_code" in df.columns:
    df = df.with_columns(
        pl.col("legs0_segments0_marketingCarrier_code").is_in(["SU", "S7", "U6"])
            .cast(pl.Int32).alias("is_major_carrier"))
else:
    df = df.with_columns(pl.lit(0).alias("is_major_carrier"))
    

time_exprs = []
for col in ("legs0_departureAt", "legs0_arrivalAt", "legs1_departureAt", "legs1_arrivalAt"):
    if col in df.columns:
        dt = pl.col(col).str.to_datetime(strict=False)
        h = dt.dt.hour().fill_null(12)
        time_segment = (h // 4)
        time_exprs.extend([
            h.alias(f"{col}_hour"),
            dt.dt.weekday().fill_null(0).alias(f"{col}_weekday"),
            (((h >= 6) & (h <= 9)) | ((h >= 17) & (h <= 20))).cast(pl.Int32).alias(f"{col}_business_time"),
            time_segment.alias(f"{col}_time_segment")
        ])
if time_exprs:
    df = df.with_columns(time_exprs)


rank_exprs = []
for col, alias in [
    ('totalPrice',           'price'),
    ('total_duration',       'duration'),
    ('avg_cabin_class',      'cabin_class'),
    ('total_fees',           'fees'),
    ('segments_count',       'segments'),
    ('taxes',                'taxes'),
    ('carrier_pop_combined', 'carrier_pop')
]:
    raw   = pl.col(col).rank().over('ranker_id')
    norm  = (raw - 1) / (pl.col('group_size') - 1)
    rank_exprs += [
        raw.alias(f'{alias}_rank'),
        norm.alias(f'{alias}_rank_norm')
    ]

df = df.with_columns(rank_exprs)

df = df.with_columns(
    ((
        pl.col('price_rank_norm') +
        pl.col('duration_rank_norm') +
        pl.col('cabin_class_rank_norm')
    ) / 3)
    .alias('combined_rank_norm')
)

features = []
for col in ['totalPrice','total_duration', 'avg_cabin_class', 'total_fees', 'taxes']:
    raw   = pl.col(col)
    rmin  = (raw / (raw.min().over('ranker_id') + 1e-6)).alias(f'{col}_ratio_min')
    delt  = (raw - raw.min().over('ranker_id')).alias(f'{col}_delta_min')
    qtl   = (raw.rank().over('ranker_id') - 1) / (pl.col('group_size') - 1)
    features += [rmin, delt, qtl.alias(f'{col}_qtl')]

df = df.with_columns(features)

price_exprs = [
    (pl.col("totalPrice") == pl.col("totalPrice").min().over("ranker_id")).cast(pl.Int32).alias("is_cheapest"),
    ((pl.col("totalPrice") - pl.col("totalPrice").median().over("ranker_id")) / 
    (pl.col("totalPrice") / pl.col("total_duration").alias("price_per_duration")))
]

df = df.with_columns(price_exprs)

df = df.with_columns(
    (
        # Policy compliance (25% weight)
        (pl.col("pricingInfo_isAccessTP") * 0.25) +
        # Direct flights (25% weight)
        (pl.col("is_direct_leg0") * 0.25) +
        # Business-hour departures/arrivals (25% weight)
        ((pl.col("legs0_departureAt_business_time") + pl.col("legs1_departureAt_business_time")) * 0.125) +
        # VIP preference for business class (25% weight)
        ((pl.col("isVip") == 1) & (pl.col("avg_cabin_class") >= 1.5)).cast(pl.Int8) * 0.25
    ).alias("business_traveler_perfect_match"),

    # Timezone diff only
    (pl.col("legs0_arrivalAt_hour") - pl.col("legs0_departureAt_hour") -
     (pl.col("legs0_duration") / 60)).alias("timezone_diff_leg0"),
)

df = df.with_columns(
    (
        (pl.col("is_one_way") == 0) &  # Round-trip
        (pl.col("legs0_arrivalAt_hour") >= 8) &  # Arrive by morning
        (pl.col("legs1_departureAt_hour") <= 18) &  # Return by evening
        (pl.col("timezone_diff_leg0").abs() < 3)   # Minimal jetlag
    ).cast(pl.Int8).alias("meeting_friendly_itinerary")
)

is_direct_cheapest_expr = (
    (pl.col("is_direct_leg0") == 1) &
    (pl.col("totalPrice") == 
        pl.when(pl.col("is_direct_leg0") == 1)
          .then(pl.col("totalPrice"))
          .min()
          .over("ranker_id")
    )
).cast(pl.Int32).fill_null(0).alias("is_direct_cheapest")

within_TP_highest_class_expr = (
    (pl.col("pricingInfo_isAccessTP") == 1) &
    (pl.col("avg_cabin_class") == 
        pl.when(pl.col("pricingInfo_isAccessTP") == 1)
          .then(pl.col("avg_cabin_class"))
           .max()
          .over("ranker_id")
    )
).cast(pl.Int32).fill_null(0).alias("within_TP_highest_class")

is_cheapest_corporate_expr = (
    (pl.col("corporateTariffCode").is_not_null()) &
    (pl.col("totalPrice") == 
        pl.when(pl.col("corporateTariffCode").is_not_null())
          .then(pl.col("totalPrice"))
          .min()
          .over("ranker_id")
    )
).cast(pl.Int32).fill_null(0).alias("is_cheapest_corporate")

is_highest_class_corporate_expr = (
    (pl.col("corporateTariffCode").is_not_null()) &
    (pl.col("avg_cabin_class") == 
        pl.when(pl.col("corporateTariffCode").is_not_null())
          .then(pl.col("avg_cabin_class"))
          .max()
          .over("ranker_id")
    )
).cast(pl.Int32).fill_null(0).alias("is_highest_class_corporate")

is_free_cancel = (
    (pl.col("miniRules0_statusInfos") == 1) &
    (pl.col("miniRules0_monetaryAmount") == 0)
).fill_null(False).alias("free_cancel")

is_free_exchange = (
    (pl.col("miniRules1_statusInfos") == 1) &
    (pl.col("miniRules1_monetaryAmount") == 0)
    ).fill_null(False).alias("free_exchange")


# --- Execute them all at once ---
df = df.with_columns(
    is_direct_cheapest_expr,
    within_TP_highest_class_expr,
    is_cheapest_corporate_expr,
    is_highest_class_corporate_expr,
    is_free_cancel,
    is_free_exchange,
)

In [ ]:
# Fill nulls
data = df.with_columns(
    [pl.col(c).fill_null(0) for c in df.select(pl.selectors.numeric()).columns] +
    [pl.col(c).fill_null("missing") for c in df.select(pl.selectors.string()).columns]
)

## Feature Selection

In [ ]:
# Categorical features
cat_features = [
    'nationality', 'searchRoute', 'corporateTariffCode',
    'bySelf', 'sex', 'companyID',
    # Leg 0 segments 0-1
    'legs0_segments0_aircraft_code', 'legs0_segments0_arrivalTo_airport_city_iata',
    'legs0_segments0_arrivalTo_airport_iata', 'legs0_segments0_departureFrom_airport_iata',
    'legs0_segments0_marketingCarrier_code', 'legs0_segments0_operatingCarrier_code',
    'legs0_segments0_flightNumber',
    'legs0_segments1_aircraft_code', 'legs0_segments1_arrivalTo_airport_city_iata',
    'legs0_segments1_arrivalTo_airport_iata', 'legs0_segments1_departureFrom_airport_iata',
    'legs0_segments1_marketingCarrier_code', 'legs0_segments1_operatingCarrier_code',
    'legs0_segments1_flightNumber',
    # Leg 1 segments 0-1
    'legs1_segments0_aircraft_code', 'legs1_segments0_arrivalTo_airport_city_iata',
    'legs1_segments0_arrivalTo_airport_iata', 'legs1_segments0_departureFrom_airport_iata',
    'legs1_segments0_marketingCarrier_code', 'legs1_segments0_operatingCarrier_code',
    'legs1_segments0_flightNumber',
    'legs1_segments1_aircraft_code', 'legs1_segments1_arrivalTo_airport_city_iata',
    'legs1_segments1_arrivalTo_airport_iata', 'legs1_segments1_departureFrom_airport_iata',
    'legs1_segments1_marketingCarrier_code', 'legs1_segments1_operatingCarrier_code',
    'legs1_segments1_flightNumber',
]

te_features = [
    'corporateTariffCode',
    # Leg 0 segments 0-1
    'legs0_segments0_aircraft_code',
    'legs0_segments0_marketingCarrier_code', 'legs0_segments0_operatingCarrier_code',
    'legs0_segments1_aircraft_code',
    'legs0_segments1_marketingCarrier_code', 'legs0_segments1_operatingCarrier_code',
    # Leg 1 segments 0-1
    'legs1_segments0_aircraft_code',
    'legs1_segments0_marketingCarrier_code', 'legs1_segments0_operatingCarrier_code',
    'legs1_segments1_aircraft_code',
    'legs1_segments1_marketingCarrier_code', 'legs1_segments1_operatingCarrier_code',
]

# Columns to exclude (uninformative or problematic)
exclude_cols = [
    'Id', 'ranker_id', 'selected', 'profileId', 'requestDate', "pricingInfo_isAccessTP",
    'legs0_departureAt', 'legs0_arrivalAt', 'legs1_departureAt', 'legs1_arrivalAt', "bySelf",
    'frequentFlyer', 'pricingInfo_passengerCount'
]


# Exclude segment 2-3 columns (>98% missing)
for leg in [0, 1]:
    for seg in [2, 3]:
        for suffix in ['aircraft_code', 'arrivalTo_airport_city_iata', 'arrivalTo_airport_iata',
                      'baggageAllowance_quantity',
                      'cabinClass', 'departureFrom_airport_iata', 'duration', 'flightNumber',
                      'marketingCarrier_code', 'operatingCarrier_code', 'seatsAvailable']:
            exclude_cols.append(f'legs{leg}_segments{seg}_{suffix}')


for leg in [0, 1]:
    for seg in [0, 1]:
        if seg == 0:
            suffixes = [
                "seatsAvailable",
            ]
        else:
            suffixes = [
                "cabinClass",
                "seatsAvailable",
                "baggageAllowance_quantity",
                "baggageAllowance_weightMeasurementType",
                "aircraft_code",
                "arrivalTo_airport_city_iata",
                "arrivalTo_airport_iata",
                "departureFrom_airport_iata",
                "flightNumber",
                "marketingCarrier_code",
                "operatingCarrier_code",
            ]
        for suffix in suffixes:
            exclude_cols.append(f"legs{leg}_segments{seg}_{suffix}")


feature_cols = [col for col in data.columns if col not in exclude_cols]
cat_features_final = [col for col in cat_features if col in feature_cols]

print(feature_cols)
print(cat_features_final)
print(f"Using {len(feature_cols)} features ({len(cat_features_final)} categorical)")

# Not used
small_model_features = [
    "totalPrice",
    "total_duration",
    "fee_rate",

    "is_one_way",
    "segments_count",
    "n_segments_leg0",
    "n_segments_leg1",
    "is_direct_leg0",
    "is_direct_leg1",
    "is_min_segments",

    "is_major_carrier",
    "avg_cabin_class",
    "has_baggage",
    "has_access_tp",
    "free_exchange",
    "free_cancel",
    "corporateTariffCode",
    "within_TP_highest_class",

    "days_to_departure",
    "legs0_departureAt_hour",
    "legs0_departureAt_weekday",
    "legs0_departureAt_business_time",
    "legs0_arrivalAt_hour",
    "legs1_departureAt_hour",

    "price_rank_norm",
    "duration_rank_norm",
    "cabin_class_rank_norm",

    "total_duration_delta_min",
    "totalPrice_delta_min",

]

X = data.select(feature_cols)
y = data.select('selected')
groups = data.select('ranker_id')

## Model Training

In [ ]:
data_final = X.with_columns([(pl.col(c).rank("dense") - 1).fill_null(-1).cast(pl.Int32) for c in cat_features_final])

n1 = 16487352 # split train to train and val (10%) in time
n2 = train.height
data_tr, data_va, data_te = data_final[:n1], data_final[n1:n2], data_final[n2:]
y_tr, y_va, y_te = y[:n1], y[n1:n2], y[n2:]
groups_tr, groups_va, groups_te = groups[:n1], groups[n1:n2], groups[n2:]

target_col = "selected" 
prior = y_tr.select(pl.col(y_tr.columns[0]).mean()).item()

df_tr = data_tr.with_columns(pl.Series(name=target_col, values=y_tr))

for c in te_features:
    if c not in df_tr.columns:
        continue

    mp = (
        df_tr.group_by(c)
             .agg([
                 pl.len().alias("cnt"),
                 pl.col(target_col).sum().alias("sum_y")
             ])
             .with_columns(
                 ((pl.col("sum_y") + 10 * prior) / (pl.col("cnt") + 10)).alias(f"{c}_te")
             )
             .select([c, f"{c}_te"])
    )

    data_tr = (data_tr.join(mp, on=c, how="left").with_columns(pl.col(f"{c}_te").fill_null(prior)))
    data_va = (data_va.join(mp, on=c, how="left").with_columns(pl.col(f"{c}_te").fill_null(prior)))
    data_te = (data_te.join(mp, on=c, how="left").with_columns(pl.col(f"{c}_te").fill_null(prior)))


y_tr = y_tr["selected"].to_numpy()
y_va = y_va["selected"].to_numpy()
y_te = y_te["selected"].to_numpy()

group_sizes_tr = groups_tr.group_by('ranker_id', maintain_order=True).agg(pl.len())['len'].to_numpy()
group_sizes_va = groups_va.group_by('ranker_id', maintain_order=True).agg(pl.len())['len'].to_numpy()
group_sizes_te = groups_te.group_by('ranker_id', maintain_order=True).agg(pl.len())['len'].to_numpy()

dtrain_xgb = xgb.DMatrix(data_tr, label=y_tr, group=group_sizes_tr)
dval_xgb   = xgb.DMatrix(data_va, label=y_va, group=group_sizes_va)
dtest_xgb  = xgb.DMatrix(data_te, label=y_te, group=group_sizes_te)

dtrain_lgb = lgb.Dataset(data_tr, label=y_tr, group=group_sizes_tr)
dval_lgb   = lgb.Dataset(data_va, label=y_va, group=group_sizes_va, reference=dtrain_lgb)
dtest_lgb  = lgb.Dataset(data_te, label=y_te, group=group_sizes_te, reference=dtrain_lgb)

In [ ]:
model_type = "ensemble"
tuning = False

In [ ]:
if tuning:

    import optuna
    import functools
    
    if model_type == "lgb":
        
        from optuna.integration import LightGBMPruningCallback
        
        def objective(trial, dtrain, dval):
            params = {
                'objective': 'lambdarank',
                'metric': 'ndcg',
                'eval_at': [3],
                'boosting_type': 'gbdt',
                'seed': RANDOM_STATE,
                'n_jobs': -1,
                'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1, log=True),
                'num_leaves': trial.suggest_int('num_leaves', 31, 150),
                'max_depth': trial.suggest_int('max_depth', 5, 10),
                'min_child_weight': trial.suggest_float('min_child_weight', 1, 10),
                'feature_fraction': trial.suggest_float('feature_fraction', 0.7, 1.0),
                'bagging_fraction': trial.suggest_float('bagging_fraction', 0.7, 1.0),
                'bagging_freq': trial.suggest_int('bagging_freq', 1, 5),
                'lambda_l2': trial.suggest_float('lambda_l2', 1e-2, 10.0, log=True),
                'verbosity': -1
            }
        
            model = lgb.train(
                params,
                dtrain,
                num_boost_round=800,  # ↓ smaller during tuning
                valid_sets=[dtrain, dval],
                valid_names=['train', 'val'],
                callbacks=[
                    lgb.early_stopping(stopping_rounds=50, verbose=False),
                    LightGBMPruningCallback(trial, "ndcg@3", valid_name="val")
                ]
            )
    
            return model.best_score['val']['ndcg@3']
        
        # --- Run the study ---
        study = optuna.create_study(direction='maximize')
        objective_with_data = functools.partial(objective, dtrain=dtrain_lgb, dval=dval_lgb)
        study.optimize(objective_with_data, n_trials=40, n_jobs=1)
    
        
        # --- Print the results ---
        print("\n--- Optuna Study Results ---")
        print(f"Number of finished trials: {len(study.trials)}")
        print("Best trial:")
        best_trial = study.best_trial
        print(f"  Value (NDCG@3): {best_trial.value:.5f}")
        print("  Best Parameters: ")
        for key, value in best_trial.params.items():
            print(f"    {key}: {value}")

    elif model_type == "xgb":

        from optuna.integration import XGBoostPruningCallback
        
        def objective(trial, dtrain, dval):
            params = {
                'objective': 'rank:pairwise',   # XGBoost ranking objective
                'eval_metric': 'ndcg@3',    # Same evaluation as LightGBM
                'tree_method': 'hist',      # Fast, memory-efficient histogram algorithm
                'seed': RANDOM_STATE,
                'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1, log=True),
                'max_depth': trial.suggest_int('max_depth', 5, 10),
                'min_child_weight': trial.suggest_float('min_child_weight', 1, 10),
                'colsample_bytree': trial.suggest_float('colsample_bytree', 0.7, 1.0),
                'subsample': trial.suggest_float('subsample', 0.7, 1.0),
                'lambda': trial.suggest_float('lambda', 1e-2, 10.0, log=True),  # L2 regularization
                'alpha': trial.suggest_float('alpha', 1e-2, 10.0, log=True),   # L1 regularization
            }
    
            # Train the model
            bst = xgb.train(
                params,
                dtrain,
                num_boost_round=800,  # Can be smaller during tuning
                evals=[(dtrain, 'train'), (dval, 'val')],
                early_stopping_rounds=50,
                verbose_eval=False,
                callbacks=[
                    XGBoostPruningCallback(trial, "val-ndcg@3")
                ]
            )
        
            # Return the best NDCG score
            return bst.best_score
        
        # --- Run the study ---
        study = optuna.create_study(direction='maximize')
        objective_with_data = functools.partial(objective, dtrain=dtrain_xgb, dval=dval_xgb)
        study.optimize(objective_with_data, n_trials=40, n_jobs=1)
        
        # --- Print the results ---
        print("\n--- Optuna Study Results ---")
        print(f"Number of finished trials: {len(study.trials)}")
        print("Best trial:")
        best_trial = study.best_trial
        print(f"  Value (NDCG@3): {best_trial.value:.5f}")
        print("  Best Parameters: ")
        for key, value in best_trial.params.items():
            print(f"    {key}: {value}")

In [ ]:
print(model_type)

if model_type == "xgb" or model_type == "ensemble":
    xgb_params = {
        'objective': 'rank:pairwise',
        'eval_metric': 'ndcg@3',
        'learning_rate': 0.06030516375230898,
        'max_depth': 9,
        'min_child_weight': 3.4021529387521423,
        'colsample_bytree': 0.7437840878139231,
        'subsample': 0.9493117941317905,
        'lambda': 2.801483598481627,
        'alpha': 0.531733951944211,
        'seed': RANDOM_STATE,
        'n_jobs': -1,
        # 'device': 'cuda'
    }
    
    # Train XGBoost model
    print("Training XGBoost model...")
    xgb_model = xgb.train(
        xgb_params,
        dtrain_xgb,
        num_boost_round=1500,
        evals=[(dtrain_xgb, 'train'), (dval_xgb, 'val')],
        early_stopping_rounds=50,
        verbose_eval=50
    )
if model_type == "lgb" or model_type == "ensemble":
    lgb_params = {
        'objective': 'lambdarank',  
        'metric': 'ndcg',
        'eval_at': [3], 
        'boosting_type': 'gbdt',
        'num_leaves': 255,  
        'max_depth': 10,
        'min_child_weight': 10,
        'bagging_fraction': 0.8, 
        'feature_fraction': 0.8,
        'lambda_l2': 10.0,
        'learning_rate': 0.05,
        'seed': RANDOM_STATE,
        'n_jobs': -1,
    }
    
    lgb_model = lgb.train(
        lgb_params,
        dtrain_lgb,
        num_boost_round=1000,
        valid_sets=[dtrain_lgb, dval_lgb],
        valid_names=['train', 'val'],
        callbacks=[lgb.log_evaluation(period=50), lgb.early_stopping(stopping_rounds=100)]
    )


# Performance analysis and visualizations

In [ ]:
if model_type == "xgb":
    # Evaluate XGBoost
    xgb_va_preds = xgb_model.predict(dval_xgb)
    xgb_hr3 = hitrate_at_3(y_va, xgb_va_preds, groups_va)
    print(f"HitRate@3: {xgb_hr3:.3f}")
else:
    # Evaluate LGBM
    lgb_va_preds = lgb_model.predict(data_va)
    lgb_hr3 = hitrate_at_3(y_va, lgb_va_preds, groups_va)
    print(f"HitRate@3: {lgb_hr3:.3f}")

In [ ]:
if model_type == "xgb":
    xgb_importance = xgb_model.get_score(importance_type='gain')
    xgb_importance_df = pl.DataFrame(
        [{'feature': k, 'importance': v} for k, v in xgb_importance.items()]
    ).sort('importance', descending=bool(1))
    print(xgb_importance_df.head(100).to_pandas().to_string())

In [ ]:
# Trying out shap analysis for lgb

if model_type == "lgb":

    import shap 
    
    X_sample = data_va.sample(100000)
    X_np = X_sample.to_numpy()
    
    explainer = shap.TreeExplainer(lgb_model)
    shap_values = explainer.shap_values(X_np)
    
    shap.summary_plot(shap_values, X_sample.to_pandas(), plot_type="bar", max_display=200)
    shap.summary_plot(shap_values, X_sample.to_pandas())
    
    
    import numpy as np
    
    threshold = 0.1

    mean_abs_shap = np.abs(shap_values).mean(axis=0)
    
    feature_names = X_sample.columns
    
    low_importance_features = []
    for i, score in enumerate(mean_abs_shap):
        if score < threshold:
            low_importance_features.append((feature_names[i], score))
    
    low_importance_features.sort(key=lambda x: x[1])
    
    if not low_importance_features:
        print(f"\n✅ All features have a mean absolute SHAP value of {threshold} or greater.")
    else:
        print(f"\nFound {len(low_importance_features)} features with a mean absolute SHAP value below {threshold}:")
        print("-" * 70)
        print(f"{'Feature Name':<45} | {'Mean ABS SHAP Value'}")
        print("-" * 70)
        for name, score in low_importance_features:
            print(f"{name:<45} | {score:.6f}")
        print("-" * 70)


In [ ]:
# Analysis of xgb hit rate for different group sizes

if model_type == "xgb":
    # Color palette
    red = (0.86, 0.08, 0.24)
    blue = (0.12, 0.56, 1.0)
    
    # Prepare data for analysis
    va_df = pl.DataFrame({
        'ranker_id': groups_va.to_numpy().flatten(),
        'pred_score': xgb_va_preds,
        'selected': y_va.flatten()
    })
    
    # Add group size and filter
    va_df = va_df.join(
        va_df.group_by('ranker_id').agg(pl.len().alias('group_size')), 
        on='ranker_id'
    ).filter(pl.col('group_size') > 10)
    
    # Calculate group size quantiles
    size_quantiles = va_df.select('ranker_id', 'group_size').unique().select(
        pl.col('group_size').quantile(0.25).alias('q25'),
        pl.col('group_size').quantile(0.50).alias('q50'),
        pl.col('group_size').quantile(0.75).alias('q75')
    ).to_dicts()[0]
    
    # Function to calculate hitrate curve efficiently
    def calculate_hitrate_curve(df, k_values):
        # Sort once and calculate all k values
        sorted_df = df.sort(["ranker_id", "pred_score"], descending=[False, True])
        return [
            sorted_df.group_by("ranker_id", maintain_order=True)
            .head(k)
            .group_by("ranker_id")
            .agg(pl.col("selected").max().alias("hit"))
            .select(pl.col("hit").mean())
            .item()
            for k in k_values
        ]
    
    # Calculate curves
    k_values = list(range(1, 21))
    curves = {
        'All groups (>10)': calculate_hitrate_curve(va_df, k_values),
        f'Small (11-{int(size_quantiles["q25"])})': calculate_hitrate_curve(
            va_df.filter(pl.col('group_size') <= size_quantiles['q25']), k_values
        ),
        f'Medium ({int(size_quantiles["q25"]+1)}-{int(size_quantiles["q75"])})': calculate_hitrate_curve(
            va_df.filter((pl.col('group_size') > size_quantiles['q25']) & 
                        (pl.col('group_size') <= size_quantiles['q75'])), k_values
        ),
        f'Large (>{int(size_quantiles["q75"])})': calculate_hitrate_curve(
            va_df.filter(pl.col('group_size') > size_quantiles['q75']), k_values
        )
    }
    
    # Calculate hitrate@3 by group size using log-scale bins
    # Create log-scale bins
    min_size = va_df['group_size'].min()
    max_size = va_df['group_size'].max()
    bins = np.logspace(np.log10(min_size), np.log10(max_size), 51)  # 51 edges = 50 bins
    
    # Calculate hitrate@3 for each ranker_id
    ranker_hr3 = (
        va_df.sort(["ranker_id", "pred_score"], descending=[False, True])
        .group_by("ranker_id", maintain_order=True)
        .agg([
            pl.col("selected").head(3).max().alias("hit_top3"),
            pl.col("group_size").first()
        ])
    )
    
    # Assign bins and calculate hitrate per bin
    bin_centers = (bins[:-1] + bins[1:]) / 2  # Geometric mean would be more accurate for log scale
    bin_indices = np.digitize(ranker_hr3['group_size'].to_numpy(), bins) - 1
    
    size_analysis = pl.DataFrame({
        'bin_idx': bin_indices,
        'bin_center': bin_centers[np.clip(bin_indices, 0, len(bin_centers)-1)],
        'hit_top3': ranker_hr3['hit_top3']
    }).group_by(['bin_idx', 'bin_center']).agg([
        pl.col('hit_top3').mean().alias('hitrate3'),
        pl.len().alias('n_groups')
    ]).filter(pl.col('n_groups') >= 3).sort('bin_center')  # At least 3 groups per bin
    
    # Create combined figure
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 4), dpi=400)
    
    # Left plot: HitRate@k curves
    # Create color gradient from blue to red for size groups
    colors = ['black']  # All groups is black
    for i in range(3):  # 3 size groups
        t = i / 2  # 0, 0.5, 1
        color = tuple(blue[j] * (1 - t) + red[j] * t for j in range(3))
        colors.append(color)
    
    for (label, hitrates), color in zip(curves.items(), colors):
        ax1.plot(k_values, hitrates, marker='o', label=label, color=color, markersize=3)
    ax1.set_xlabel('k (top-k predictions)')
    ax1.set_ylabel('HitRate@k')
    ax1.set_title('HitRate@k by Group Size')
    ax1.legend(fontsize=8)
    ax1.grid(True, alpha=0.3)
    ax1.set_xlim(0, 21)
    ax1.set_ylim(-0.025, 1.025)
    
    # Right plot: HitRate@3 vs Group Size (log scale)
    ax2.scatter(size_analysis['bin_center'], size_analysis['hitrate3'], s=30, alpha=0.6, color=blue)
    ax2.set_xlabel('Group Size')
    ax2.set_ylabel('HitRate@3')
    ax2.set_title('HitRate@3 vs Group Size')
    ax2.set_xscale('log')
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

    # Summary
    print(f"HitRate@1: {curves['All groups (>10)'][0]:.3f}")
    print(f"HitRate@3: {curves['All groups (>10)'][2]:.3f}")
    print(f"HitRate@5: {curves['All groups (>10)'][4]:.3f}")
    print(f"HitRate@10: {curves['All groups (>10)'][9]:.3f}")


In [ ]:
if model_type == "xgb":
    submission_xgb = (
        test.select(['Id', 'ranker_id'])
        .with_columns(pl.Series('pred_score', xgb_model.predict(dtest_xgb)))
        .with_columns(
            pl.col('pred_score')
            .rank(method='ordinal', descending=True)
            .over('ranker_id')
            .cast(pl.Int32)
            .alias('selected')
        )
        .select(['Id', 'ranker_id', 'selected'])
    )
    submission_xgb.write_csv('submission.csv')
elif model_type == "lgb":
    submission_lgb = (
    test.select(['Id', 'ranker_id'])
    .with_columns(pl.Series('pred_score', lgb_model.predict(data_te)))
    .with_columns(
        pl.col('pred_score')
        .rank(method='ordinal', descending=True)
        .over('ranker_id')
        .cast(pl.Int32)
        .alias('selected')
        )
    .select(['Id', 'ranker_id', 'selected'])
    )
    submission_lgb.write_csv('submission.csv')
elif model_type == "ensemble":
    alpha = 0.6
    submission_ens = (
    test.select(['Id', 'ranker_id'])
    .with_columns(pl.Series('pred_score', (alpha * lgb_model.predict(data_te)) + ((1 - alpha) * xgb_model.predict(dtest_xgb))))
    .with_columns(
        pl.col('pred_score')
        .rank(method='ordinal', descending=True)
        .over('ranker_id')
        .cast(pl.Int32)
        .alias('selected')
        )
    .select(['Id', 'ranker_id', 'selected'])
    )
    submission_ens.write_csv('submission.csv')